In [1]:
import os
import cv2 as cv
import numpy as np
import pandas as pd

In [2]:
def getcell(matrix, starti, startj):
    ret = []
    for i in range(4 * starti, 4 * starti + 10):
        tmp = []
        for j in range(4 * startj, 4 * startj + 10):
            tmp.append(matrix[i][j])
        ret.append(tmp)
    return ret

In [3]:
def HOF(magnitude, angle):
    ret = []
    unitdegree = np.pi / 4
    for i in range(int((len(magnitude) - 10) / 4) + 1):
        for j in range(int((len(magnitude[0]) - 10) / 4) + 1):
            cellmagnitude = getcell(magnitude, i, j)
            cellangle = getcell(angle, i, j)
            mydict = {new_list: [] for new_list in range(8)}
            for subi in range(len(cellmagnitude)):
                for subj in range(len(cellmagnitude[0])):
#                     if subi >= len(cellangle) or subj >= len(cellangle[0]):
#                         print(subi, subj)
#                         print(len(cellangle), len(cellangle[0]))
                    num_rad = int(cellangle[subi][subj] // unitdegree)
                    if num_rad == 8:
                        num_rad = 7
                    mydict[num_rad].append(cellmagnitude[subi][subj])
            assert(len(mydict[1]) + len(mydict[2]) + len(mydict[3]) + len(mydict[4]) + len(mydict[5]) + len(mydict[6]) + len(mydict[7]) + len(mydict[0]) == 100)
            ret.append(mydict)
    return ret


In [4]:
def normalize(mylist):
    maxv = [0] * 8
    for mydict in mylist:
        for i in range(len(maxv)):
            if mydict[i]:
                maxv[i] = max(maxv[i], max(mydict[i]))
    for mydict in mylist:
        for i in range(len(maxv)):
            for j in range(len(mydict[i])):
                mydict[i][j] = mydict[i][j]/maxv[i]
    print('maxv', maxv)
    
def vec_to_hist(myvec):
    ret = [0] * 10
    for ele in myvec:
        index = int(ele // 0.1)
        ret[index] += 1
    return ret
def to_hist(mylist):
    ret = []
    normalize(mylist)
    for mydict in mylist:
        tmpret = []
        for key in mydict:
            tmpret =tmpret + vec_to_hist(mydict[key])
        ret.append(tmpret)
        assert(sum(tmpret) == 100)
    return ret

In [ ]:
cwd = os.getcwd()
subfolders = [dir for dir in os.listdir() if dir[:5] == "Train"]
mylist = []
for subfolder in sorted(subfolders):
    path = cwd + '/' + subfolder
    prev = None
    for img in sorted(os.listdir(path)):
        if img[-3:] != 'tif':
            continue
        frame = cv.imread(path + '/' + img, 0)
        if prev is not None:
            flow = cv.calcOpticalFlowFarneback(prev, frame, None, 0.5, 3, 15, 3, 5, 1.2, 0)
            magnitude, angle = cv.cartToPolar(flow[..., 0], flow[..., 1])  # return angle in radians
            mylist = mylist + HOF(magnitude, angle)
            # flow_color = flow_vis.flow_to_color(flow, convert_to_bgr=False)
            # cv.imshow('flow', flow_color)
            # cv.waitKey(1)
        # cv.imshow('image', frame)
        # cv.waitKey(1)

        prev = frame

In [7]:
histogram = to_hist(mylist)

maxv [7.4396048, 5.681708, 6.0392303, 6.0397787, 4.9539485, 4.7482467, 6.101315, 6.857308]


In [8]:
histogram = np.array(histogram)
vec_distance = []
# for i in range(len(histogram)):
#     for j in range(i+1, len(histogram)):
#         vec_distance.append(np.linalg.norm(histogram[i]-histogram[j]))
# mean_distance = 5 * sum(vec_distance)/len(vec_distance)
for i in range(1, len(histogram)):
    vec_distance.append(np.linalg.norm(histogram[i]-histogram[i-1]))

In [9]:
def construct_basic(h, epsilon, alpha):
    cs = []
    fs = []
    pcs = []
    for j in range(len(h)):
        if j == 0:
            cs.append(pd.Series(h[j]))
            fs.append(1)
            pcs.append(1)
        else:
            addvec = False
            for ci in cs:
                dist = np.linalg.norm(ci-h[j])
                if dist > epsilon:
                    addvec = True
                    break
            if addvec:
                cs.append(h[j])
                fs.append(1)
                pcs.append(1/len(h))
            else:
                sum_inv_dist = 0
                for ci in cs:
                    sum_inv_dist += 1/np.linalg.norm(ci-h[j])
                for i in range(len(cs)):
                    wi = 1/sum_inv_dist * 1/np.linalg.norm(cs[i]-h[j])
                    cs[i] = (fs[i]*cs[i]+wi*h[j])/(fs[i]+wi)
                    fs[i] += wi
                    pcs[i] = fs[i]/len(h)
    assert(abs(sum(pcs) - 1)<0.01)
    assert(abs(sum(fs)-len(h))<1)
    def pruning():
        deleted = set()
        for i in range(len(cs)):
            if i in deleted:
                continue
            for j in range(i+1, min(i+15000000,len(cs))):
                if np.linalg.norm(cs[i]-cs[j]) < alpha*epsilon and pcs[j] < 0.05 and j not in deleted:
                    deleted.add(j)
                    cs[i] = [(cs[i][k]*fs[i]+cs[j][k]*fs[j])/(fs[i]+fs[j]) for k in range(len(cs[0]))]
                    fs[i] += fs[j]
                    pcs[i] += pcs[j]
        ret = []
        for i in range(len(cs)):
            if i not in deleted:
                ret.append(cs[i])
        return ret
    ret = []
    for i in range(1):
        ret = pruning()
    return ret
        
                    

In [10]:
mean_distance = 2*sum(vec_distance)/len(vec_distance)
mean_distance

basic_vec = construct_basic(histogram, mean_distance, 1)

In [30]:
anomaly = open("/home/dj1057/3001/UCSDped1/UCSDped1.txt","r") 
anomaly = anomaly.read()
anomaly = anomaly.split('\n')
anomaly.pop()
len(anomaly)

36

In [31]:
folder = os.listdir('/home/dj1057/3001/UCSDped1/Test/Test001')
testlist = []
prev = None
for img in sorted(folder):
    if img[-3:] != 'tif':
        continue
    frame = cv.imread(path + '/' + img, 0)
    if prev is not None:
        flow = cv.calcOpticalFlowFarneback(prev, frame, None, 0.5, 3, 15, 3, 5, 1.2, 0)
        magnitude, angle = cv.cartToPolar(flow[..., 0], flow[..., 1])  # return angle in radians
        testlist = testlist + HOF(magnitude, angle)

    prev = frame
testhistogram = to_hist(testlist)

maxv [4.578386, 3.1334698, 2.0328202, 3.1095643, 2.248727, 1.3622833, 2.2275581, 4.723623]


In [32]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import ConstantKernel, RBF

rbf = RBF(length_scale=10.0)
gpr = GaussianProcessRegressor(kernel=rbf, alpha=0.4**2)


gpr.fit(basic_vec, np.zeros(len(basic_vec)))

GaussianProcessRegressor(alpha=0.16000000000000003, copy_X_train=True,
                         kernel=RBF(length_scale=1e+03), n_restarts_optimizer=0,
                         normalize_y=False, optimizer='fmin_l_bfgs_b',
                         random_state=None)

In [33]:
mu = gpr.predict(testhistogram)
